## Arsitektur Model Transformer

                        +-----------------------+
                        |                       |
                        |      Input Layer      |
                        | (text_input: shape)   |
                        |                       |
                        +-----------+-----------+
                                    |
                                    v
                        +-----------+-----------+
                        |                       |
                        |   Embedding Layer     |
                        |                       |
                        +-----------+-----------+
                                    |
                                    v
                        +-----------+-----------+
                        |                       |
                        | Multi-Head Attention  |
                        |                       |
                        +-----------+-----------+
                                    |
                                    v
                        +-----------+-----------+
                        |                       |
                        |  Layer Normalization  |
                        |                       |
                        +-----------+-----------+
                                    |
                                    v
                        +-----------+-----------+
                        |                       |
                        |   Feed-Forward Layer  |
                        |                       |
                        +-----------+-----------+
                                    |
                                    v
                        +-----------+-----------+
                        |                       |
                        |  Layer Normalization  |
                        |                       |
                        +-----------+-----------+
                                    |
                                    v
                        +-----------+-----------+
                        |                       |
                        |       Dropout         |
                        |                       |
                        +-----------+-----------+
                                    |
                                    v
                        +-----------+-----------+
                        |                       |
                        |     Output Layer      |
                        |                       |
                        +-----------------------+


#### Penjelasan Mengapa Memakai Model Ini

Kemampuan Self-Attention:
- Transformer menggunakan mekanisme self-attention yang memungkinkan model untuk memperhatikan semua kata dalam urutan input secara bersamaan. Ini membuatnya sangat efektif dalam menangkap dependensi jarak jauh dan hubungan kontekstual dalam teks.

Pelatihan Paralel:
- Tidak seperti model RNN/LSTM yang memproses data secara berurutan, Transformer dapat memproses seluruh urutan input secara paralel. Ini membuat pelatihan lebih cepat dan lebih efisien, terutama dengan dataset besar.

Fleksibilitas:
- Transformer dapat dengan mudah diskalakan dan dimodifikasi untuk berbagai tugas NLP, termasuk terjemahan mesin, pemahaman bacaan, dan generasi teks. Model ini dapat digunakan untuk tugas-tugas pemahaman konteks yang kompleks.

Kinerja Terbaik dalam Berbagai Tugas NLP:
- Transformer telah menunjukkan kinerja yang unggul dalam berbagai benchmark dan tugas NLP. Arsitektur ini telah menjadi standar emas untuk banyak aplikasi NLP.

#### Keunggulan Model Transformer

Pemahaman Konteks yang Baik:

- Kemampuan self-attention memungkinkan model untuk memahami konteks setiap kata dalam kalimat secara lebih baik dibandingkan dengan model sekuensial tradisional.

Skalabilitas:
- Model Transformer dapat dengan mudah ditingkatkan dengan menambahkan lebih banyak layer dan unit tanpa mempengaruhi kinerja pelatihan secara signifikan.

Generalisasi yang Lebih Baik:
- Dengan menangkap dependensi jarak jauh dalam teks, Transformer cenderung memiliki kemampuan generalisasi yang lebih baik, membuatnya efektif dalam menangani berbagai jenis teks.

#### Diagram Arsitektur yang Lebih Rinci

                               +-----------------------+
                               |      Input Layer      |
                               | (text_input: shape)   |
                               +-----------+-----------+
                                           |
                                           v
                               +-----------+-----------+
                               |   Embedding Layer     |
                               |                       |
                               +-----------+-----------+
                                           |
                                           v
                +--------------------------+--------------------------+
                |                         |                          |
                |  Multi-Head Attention   |  Multi-Head Attention    |
                | (Self-Attention)        | (Cross-Attention)        |
                +-----------+-------------+-------------+------------+
                            |                           |
                            v                           v
                +-----------+---------------------------+------------+
                |           |         Layer Normalization            |
                +-----------+---------------------------+------------+
                            |                           |
                            v                           v
                +-----------+-------------+-------------+------------+
                |  Feed-Forward Layer     |  Feed-Forward Layer      |
                +-----------+-------------+-------------+------------+
                            |                           |
                            v                           v
                +-----------+---------------------------+------------+
                |           |         Layer Normalization            |
                +-----------+---------------------------+------------+
                            |                           |
                            v                           v
                +-----------+---------------------------+------------+
                |                       Dropout                       |
                +-----------+---------------------------+------------+
                                           |
                                           v
                               +-----------+-----------+
                               |       Output Layer    |
                               +-----------------------+


### Langkah 1: Mengimpor dan Memverifikasi GPU

In [ ]:
import tensorflow as tf

# Verifikasi GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if len(tf.config.list_physical_devices('GPU')) > 0:
    tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)


### Langkah 2: Fungsi untuk Memuat dan Memproses Data

In [ ]:
import json
import os

# Fungsi untuk memuat file JSON dari folder
def load_json_files(data_folder):
    data = []
    for filename in os.listdir(data_folder):
        if filename.endswith(".json"):
            file_path = os.path.join(data_folder, filename)
            try:
                with open(file_path, 'r') as file:
                    content = json.load(file)
                    if content:  # Check if file is not empty
                        data.append(content)
            except json.JSONDecodeError:
                print(f"Skipping file {filename} due to JSON decoding error.")
            except Exception as e:
                print(f"Skipping file {filename} due to unexpected error: {e}")
    return data

# Fungsi untuk memproses data menjadi teks dan jawaban
def preprocess_data(data):
    texts = []
    answers = []
    for article in data:
        if 'konten' in article:
            texts.append(article['konten'])
            # Asumsikan bahwa jawaban adalah konten itu sendiri (untuk latihan ini)
            answers.append(article['konten'])
    return texts, answers


### Langkah 3: Tokenisasi dan Vektorisasi Data

In [ ]:
# Fungsi untuk tokenisasi dan vektorisasi teks dan jawaban
def tokenize_and_vectorize(texts, answers, max_vocab_size=10000, max_seq_length=100):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_vocab_size, oov_token='<UNK>')
    tokenizer.fit_on_texts(texts + answers)
    
    text_sequences = tokenizer.texts_to_sequences(texts)
    answer_sequences = tokenizer.texts_to_sequences(answers)
    
    # Verifikasi distribusi token <UNK>
    unk_texts = sum([seq.count(1) for seq in text_sequences]) / len(text_sequences)
    unk_answers = sum([seq.count(1) for seq in answer_sequences]) / len(answer_sequences)
    print(f"Proportion of <UNK> tokens in texts: {unk_texts:.2f}")
    print(f"Proportion of <UNK> tokens in answers: {unk_answers:.2f}")

    padded_texts = tf.keras.preprocessing.sequence.pad_sequences(text_sequences, maxlen=max_seq_length, padding='post')
    padded_answers = tf.keras.preprocessing.sequence.pad_sequences(answer_sequences, maxlen=max_seq_length, padding='post')
    
    return padded_texts, padded_answers, tokenizer


### Langkah 4: Membuat Pipeline Data

In [ ]:
# Fungsi untuk membuat pipeline data
def create_data_pipeline(padded_texts, padded_answers, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices(((padded_texts, padded_answers), padded_answers))
    dataset = dataset.shuffle(buffer_size=len(padded_texts))
    dataset = dataset.batch(batch_size)
    return dataset


### Langkah 5: Membuat Model Transformer

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Dense, LayerNormalization, MultiHeadAttention, Dropout
from tensorflow.keras.models import Model

def transformer_encoder(vocab_size, num_heads, ff_dim, num_layers, embedding_dim, max_seq_length):
    text_input = Input(shape=(max_seq_length,), name='text_input')
    answer_input = Input(shape=(max_seq_length,), name='answer_input')
    
    embedding_layer = Embedding(vocab_size, embedding_dim)
    text_embedding = embedding_layer(text_input)
    answer_embedding = embedding_layer(answer_input)
    
    for _ in range(num_layers):
        # Encoder
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(text_embedding, text_embedding)
        attn_output = Dropout(0.1)(attn_output)
        out1 = LayerNormalization(epsilon=1e-6)(text_embedding + attn_output)
        
        ffn_output = Dense(ff_dim, activation='relu')(out1)
        ffn_output = Dense(embedding_dim)(ffn_output)
        ffn_output = Dropout(0.1)(ffn_output)
        text_embedding = LayerNormalization(epsilon=1e-6)(out1 + ffn_output)
        
        # Decoder
        dec_attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(answer_embedding, text_embedding)
        dec_attn_output = Dropout(0.1)(dec_attn_output)
        out2 = LayerNormalization(epsilon=1e-6)(answer_embedding + dec_attn_output)
        
        ffn_output = Dense(ff_dim, activation='relu')(out2)
        ffn_output = Dense(embedding_dim)(ffn_output)
        ffn_output = Dropout(0.1)(ffn_output)
        answer_embedding = LayerNormalization(epsilon=1e-6)(out2 + ffn_output)
    
    output = Dense(vocab_size, activation='softmax')(answer_embedding)
    
    model = Model(inputs=[text_input, answer_input], outputs=output)
    return model

# Membuat model dengan parameter yang ditingkatkan
vocab_size = 10000
num_heads = 4  # Kurangi jumlah heads
ff_dim = 512  # Kurangi dimensi feed-forward
num_layers = 4  # Kurangi jumlah lapisan
embedding_dim = 256  # Kurangi dimensi embedding
max_seq_length = 100

transformer_model = transformer_encoder(vocab_size, num_heads, ff_dim, num_layers, embedding_dim, max_seq_length)


### Langkah 6: Melatih Model

In [ ]:
# Load and preprocess data
folder_path = './Dataset/generative_dataset'
data = load_json_files(folder_path)
texts, answers = preprocess_data(data)
padded_texts, padded_answers, tokenizer = tokenize_and_vectorize(texts, answers)

In [ ]:

# Create data pipeline
batch_size = 32
data_pipeline = create_data_pipeline(padded_texts, padded_answers, batch_size)

# Compile the model
transformer_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Melatih model dengan early stopping dan model checkpoint
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('transformer_model.h5', save_best_only=True)

# Melatih model
epochs = 50
history = transformer_model.fit(data_pipeline, epochs=epochs, callbacks=[early_stopping, model_checkpoint])


# Menyimpan model setelah pelatihan
transformer_model.save('./saved_model/saved_notebook_04/transformer_model.h5')

# Menyimpan tokenizer
import pickle

with open('./saved_model/saved_notebook_04/tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)


### Langkah 7: Visualisasi Model dan Hasil Pelatihan

In [ ]:
import matplotlib.pyplot as plt

# Plotting model architecture
tf.keras.utils.plot_model(transformer_model, to_file='transformer_model_architecture.png', show_shapes=True, show_layer_names=True)

# Visualisasi loss dan akurasi
def plot_training_history(history):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.title('Training Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.title('Training Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

# Menampilkan hasil pelatihan
plot_training_history(history)


### Langkah 8: Model Summary

In [ ]:
# Menampilkan arsitektur model
transformer_model.summary()


### Langkah 9: Memuat Model dan Tokenizer, dan Fungsi untuk Menghasilkan Output

In [ ]:
# Memuat model yang telah disimpan
transformer_model = tf.keras.models.load_model('./saved_model/saved_notebook_04/transformer_model.h5')

# Memuat tokenizer
with open('./saved_model/saved_notebook_04/tokenizer.pkl', 'rb') as file:
    tokenizer = pickle.load(file)

# Fungsi untuk menghasilkan teks dari input teks
def generate_text(input_text):
    input_seq = tokenizer.texts_to_sequences([input_text])
    padded_input = tf.keras.preprocessing.sequence.pad_sequences(input_seq, maxlen=100, padding='post')

    # Inisialisasi jawaban dengan urutan kosong
    decoded_sentence = []

    for _ in range(100):
        padded_answer = tf.keras.preprocessing.sequence.pad_sequences([decoded_sentence], maxlen=100, padding='post')

        # Menggunakan model untuk memprediksi output
        predicted_seq = transformer_model.predict([padded_input, padded_answer])

        # Mendapatkan token yang diprediksi
        predicted_token = predicted_seq.argmax(axis=-1)[0, len(decoded_sentence)]
        decoded_sentence.append(predicted_token)

    # Mengkonversi token menjadi teks
    generated_text = tokenizer.sequences_to_texts([decoded_sentence])[0]
    return generated_text

# Fungsi untuk menerima input dan menghasilkan output
def interactive_text_generation():
    while True:
        user_input = input("Masukkan pertanyaan atau 'exit' untuk keluar: ")
        if user_input.lower() == 'exit':
            break
        generated_text = generate_text(user_input)
        print(f"Teks yang dihasilkan:\n{generated_text}")

# Memulai interaksi
interactive_text_generation()

### Cell 1: Mengimpor dan Memverifikasi GPU

In [ ]:
import tensorflow as tf

# Verifikasi GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if len(tf.config.list_physical_devices('GPU')) > 0:
    tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)


### Cell 2: Fungsi untuk Memuat dan Memproses Data

In [ ]:
import json
import os

# Fungsi untuk memuat file JSON dari folder
def load_json_files(data_folder):
    data = []
    for filename in os.listdir(data_folder):
        if filename.endswith(".json"):
            file_path = os.path.join(data_folder, filename)
            try:
                with open(file_path, 'r') as file:
                    content = json.load(file)
                    if content:  # Check if file is not empty
                        data.append(content)
            except json.JSONDecodeError:
                print(f"Skipping file {filename} due to JSON decoding error.")
            except Exception as e:
                print(f"Skipping file {filename} due to unexpected error: {e}")
    return data

# Fungsi untuk memproses data menjadi teks dan jawaban
def preprocess_data(data):
    texts = []
    answers = []
    for article in data:
        if 'konten' in article:
            texts.append(article['konten'])
            # Asumsikan bahwa jawaban adalah konten itu sendiri (untuk latihan ini)
            answers.append(article['konten'])
    return texts, answers


### Cell 3: Tokenisasi dan Vektorisasi Data

In [ ]:
# Fungsi untuk tokenisasi dan vektorisasi teks dan jawaban
def tokenize_and_vectorize(texts, answers, max_vocab_size=10000, max_seq_length=100):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_vocab_size, oov_token='<UNK>')
    tokenizer.fit_on_texts(texts + answers)
    
    text_sequences = tokenizer.texts_to_sequences(texts)
    answer_sequences = tokenizer.texts_to_sequences(answers)
    
    # Verifikasi distribusi token <UNK>
    unk_texts = sum([seq.count(1) for seq in text_sequences]) / len(text_sequences)
    unk_answers = sum([seq.count(1) for seq in answer_sequences]) / len(answer_sequences)
    print(f"Proportion of <UNK> tokens in texts: {unk_texts:.2f}")
    print(f"Proportion of <UNK> tokens in answers: {unk_answers:.2f}")

    padded_texts = tf.keras.preprocessing.sequence.pad_sequences(text_sequences, maxlen=max_seq_length, padding='post')
    padded_answers = tf.keras.preprocessing.sequence.pad_sequences(answer_sequences, maxlen=max_seq_length, padding='post')
    
    return padded_texts, padded_answers, tokenizer


### Cell 4: Membuat Pipeline Data

In [ ]:
# Fungsi untuk membuat pipeline data
def create_data_pipeline(padded_texts, padded_answers, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices(((padded_texts, padded_answers), padded_answers))
    dataset = dataset.shuffle(buffer_size=len(padded_texts))
    dataset = dataset.batch(batch_size)
    return dataset


### Cell 5: Membuat Model Transformer

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Dense, LayerNormalization, MultiHeadAttention, Dropout
from tensorflow.keras.models import Model

def transformer_encoder(vocab_size, num_heads, ff_dim, num_layers, embedding_dim, max_seq_length):
    text_input = Input(shape=(max_seq_length,), name='text_input')
    answer_input = Input(shape=(max_seq_length,), name='answer_input')
    
    embedding_layer = Embedding(vocab_size, embedding_dim)
    text_embedding = embedding_layer(text_input)
    answer_embedding = embedding_layer(answer_input)
    
    for _ in range(num_layers):
        # Encoder
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(text_embedding, text_embedding)
        attn_output = Dropout(0.2)(attn_output)
        out1 = LayerNormalization(epsilon=1e-6)(text_embedding + attn_output)
        
        ffn_output = Dense(ff_dim, activation='relu')(out1)
        ffn_output = Dense(embedding_dim)(ffn_output)
        ffn_output = Dropout(0.2)(ffn_output)
        text_embedding = LayerNormalization(epsilon=1e-6)(out1 + ffn_output)
        
        # Decoder
        dec_attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(answer_embedding, text_embedding)
        dec_attn_output = Dropout(0.2)(dec_attn_output)
        out2 = LayerNormalization(epsilon=1e-6)(answer_embedding + dec_attn_output)
        
        ffn_output = Dense(ff_dim, activation='relu')(out2)
        ffn_output = Dense(embedding_dim)(ffn_output)
        ffn_output = Dropout(0.2)(ffn_output)
        answer_embedding = LayerNormalization(epsilon=1e-6)(out2 + ffn_output)
    
    output = Dense(vocab_size, activation='softmax')(answer_embedding)
    
    model = Model(inputs=[text_input, answer_input], outputs=output)
    return model

# Membuat model dengan parameter yang ditingkatkan
vocab_size = 10000
num_heads = 4  # Kurangi jumlah heads
ff_dim = 512  # Kurangi dimensi feed-forward
num_layers = 4  # Kurangi jumlah lapisan
embedding_dim = 256  # Kurangi dimensi embedding
max_seq_length = 100

transformer_model = transformer_encoder(vocab_size, num_heads, ff_dim, num_layers, embedding_dim, max_seq_length)


### Cell 6: Membagi Data Menjadi Pelatihan dan Validasi

In [ ]:
from sklearn.model_selection import train_test_split

# Load and preprocess data
folder_path = './Dataset/generative_dataset'
data = load_json_files(folder_path)
texts, answers = preprocess_data(data)
padded_texts, padded_answers, tokenizer = tokenize_and_vectorize(texts, answers)

# Membagi data menjadi pelatihan dan validasi
padded_texts_train, padded_texts_val, padded_answers_train, padded_answers_val = train_test_split(
    padded_texts, padded_answers, test_size=0.2, random_state=42)


### Cell 7: Membuat Pipeline Data untuk Pelatihan dan Validasi

In [ ]:
# Membuat pipeline data untuk pelatihan dan validasi
batch_size = 16  # Kurangi ukuran batch untuk mengurangi penggunaan memori
train_data_pipeline = create_data_pipeline(padded_texts_train, padded_answers_train, batch_size)
val_data_pipeline = create_data_pipeline(padded_texts_val, padded_answers_val, batch_size)


### Cell 8: Melatih Model dengan Data Validasi

In [ ]:
# Compile the model
transformer_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Melatih model dengan early stopping dan model checkpoint
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('./saved_model/saved_notebook_04/transformer_model.h5', save_best_only=True, monitor='val_loss')

# Melatih model
epochs = 50
history = transformer_model.fit(
    train_data_pipeline, 
    epochs=epochs, 
    validation_data=val_data_pipeline, 
    callbacks=[early_stopping, model_checkpoint])

# Menyimpan tokenizer
import pickle

with open('./saved_model/saved_notebook_04/tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)

### Cell 9: Evaluasi Model

In [ ]:
import matplotlib.pyplot as plt

# Visualisasi loss dan akurasi
def plot_training_history(history):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

# Menampilkan hasil pelatihan
plot_training_history(history)


In [ ]:
# Menampilkan arsitektur model
transformer_model.summary()

### Cell 10: Memuat Model dan Tokenizer, dan Fungsi untuk Menghasilkan Output

In [ ]:
# Memuat model yang telah disimpan
transformer_model = tf.keras.models.load_model('./saved_model/saved_notebook_04/transformer_model.h5')

# Memuat tokenizer
with open('./saved_model/saved_notebook_04/tokenizer.pkl', 'rb') as file:
    tokenizer = pickle.load(file)

# Fungsi untuk menghasilkan teks dari input teks
def generate_text(input_text):
    input_seq = tokenizer.texts_to_sequences([input_text])
    padded_input = tf.keras.preprocessing.sequence.pad_sequences(input_seq, maxlen=100, padding='post')

    # Inisialisasi jawaban dengan urutan kosong
    decoded_sentence = []

    for _ in range(100):
        padded_answer = tf.keras.preprocessing.sequence.pad_sequences([decoded_sentence], maxlen=100, padding='post')

        # Menggunakan model untuk memprediksi output
        predicted_seq = transformer_model.predict([padded_input, padded_answer])

        # Mendapatkan token yang diprediksi
        predicted_token = predicted_seq.argmax(axis=-1)[0, len(decoded_sentence)]
        decoded_sentence.append(predicted_token)

    # Mengkonversi token menjadi teks
    generated_text = tokenizer.sequences_to_texts([decoded_sentence])[0]
    return generated_text

# Fungsi untuk menerima input dan menghasilkan output
def interactive_text_generation():
    while True:
        user_input = input("Masukkan pertanyaan atau 'exit' untuk keluar: ")
        if user_input.lower() == 'exit':
            break
        generated_text = generate_text(user_input)
        print(f"Teks yang dihasilkan:\n{generated_text}")

# Memulai interaksi
interactive_text_generation()


## Rangkuman Model Transformer

#### Model Transformer yang kita bangun menggunakan arsitektur yang terdiri dari beberapa komponen penting:

Input Layers:

- Dua input layers: text_input dan answer_input, masing-masing dengan bentuk (max_seq_length,).

Embedding Layer:

- Kedua input diteruskan melalui embedding layer yang sama dengan dimensi embedding embedding_dim.

Transformer Layers:

Lapisan-lapisan Transformer yang terdiri dari beberapa komponen utama:
- Multi-Head Attention: Mechanisme self-attention dan cross-attention.
- Layer Normalization: Normalisasi setelah penjumlahan residual.
Feed-Forward Networks (FFN): Dua dense layers dengan aktivasi ReLU di antara keduanya.
- Dropout: Mencegah overfitting dengan dropout layers setelah multi-head attention dan FFN.

Output Layer:

- Dense layer dengan aktivasi softmax untuk menghasilkan distribusi probabilitas atas vocabulary.

#### Saran Peningkatan Kedepannya


Hyperparameter Tuning:

- Eksperimen dengan berbagai kombinasi hyperparameter seperti jumlah heads, dimensi feed-forward, jumlah lapisan, dan dimensi embedding.
- Gunakan alat seperti Optuna atau Keras Tuner untuk melakukan pencarian hyperparameter secara otomatis.

Preprocessing Lebih Lanjut:
- Tambahkan langkah preprocessing tambahan jika diperlukan, seperti stemming atau lemmatization, jika data memiliki banyak bentuk kata yang berbeda.

Augmentasi Data:
- Gunakan teknik augmentasi data seperti penggantian sinonim, penghapusan kata acak, atau penambahan kata acak untuk meningkatkan jumlah dan variasi data pelatihan.

Model Ensembling:
- Gunakan teknik ensembling dengan menggabungkan beberapa model Transformer untuk meningkatkan kinerja secara keseluruhan.

Transfer Learning:
- Pertimbangkan menggunakan model pralatih seperti BERT, GPT, atau T5 sebagai dasar dan fine-tuning dengan dataset spesifik Anda.

Peningkatan Infrastruktur:
- Pertimbangkan menggunakan GPU yang lebih kuat atau cluster GPU untuk mempercepat pelatihan model, terutama dengan dataset besar dan model yang lebih kompleks.

Regularisasi Lebih Lanjut:
- Eksperimen dengan teknik regularisasi tambahan seperti weight decay atau lebih banyak dropout untuk mencegah overfitting.

Evaluasi dengan Metric Berbeda:
- Selain accuracy dan loss, pertimbangkan menggunakan metrik lain seperti F1 score, BLEU score, atau ROUGE score untuk evaluasi kinerja model dalam tugas generasi teks.